In [101]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
import xgboost as xgb
import optuna
from loguru import logger
from datetime import timedelta
from mlflow.models.signature import infer_signature
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [130]:
MLFLOW_EXPERIMENT_NAME = "mobile-item-recommendation"

# Fix MLflow trash directory issue
import os
mlruns_path = os.path.join(os.getcwd(), "mlruns")
trash_path = os.path.join(mlruns_path, ".trash")
os.makedirs(trash_path, exist_ok=True)

# Optionally connect to Docker MLflow server (uncomment if running in Docker)
mlflow.set_tracking_uri("http://localhost:5000")
# mlflow.set_tracking_uri("/Users/duc.tran/mobile-item-recommendation/notebooks/mlruns")
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

<Experiment: artifact_location='file:///app/mlflow/artifacts/1', creation_time=1764185381721, experiment_id='1', last_update_time=1764185381721, lifecycle_stage='active', name='mobile-item-recommendation', tags={}>

In [107]:
import os
print(os.getcwd())

/Users/duc.tran/mobile-item-recommendation/notebooks


In [111]:
DATA_PATH = "/Users/duc.tran/mobile-item-recommendation/data/raw"
RAW_USER_DATA_PATH = os.path.join(DATA_PATH, "tianchi_fresh_comp_train_user_online_partA.txt")
RAW_ITEM_DATA_PATH = os.path.join(DATA_PATH, "tianchi_fresh_comp_train_item_online.txt")
OUTPUT_FILE = os.path.join(DATA_PATH, "items.parquet")
SAMPLE_RATE = 0.1

In [24]:
def create_sample_dataset(
    RAW_ITEM_FILE=RAW_ITEM_DATA_PATH, 
    RAW_USER_FILE=RAW_USER_DATA_PATH):

    print(f"Reading raw data (Sample Rate: {SAMPLE_RATE*100}%)...")
    
    # 1. Load the Item Subset first (We strictly need these items)
    df_item = pd.read_csv(
        RAW_ITEM_FILE, 
        header=None, 
        names=['item_id', 'item_geohash', 'item_category'],
        sep='\t',
        dtype={'item_id': 'int32', 'item_category': 'int32'}
    )
    valid_items = set(df_item['item_id'].unique())
    print(f"Item Subset Size: {len(valid_items)}")

    # 2. Read User Data in Chunks
    chunksize = 1_000_000
    chunks = []
    
    # We will select a random set of users to keep
    # But first, we need to know which users exist. 
    # Since we can't read all users easily, we filter on the fly.
    # A hash-based sampling is deterministic and efficient.
    
    for chunk in pd.read_csv(
        RAW_USER_FILE, 
        header=None,
        names=['user_id', 'item_id', 'behavior_type', 'user_geohash', 'item_category', 'time'],
        sep='\t',
        dtype={'user_id': 'int32', 'item_id': 'int32', 'behavior_type': 'int8', 'item_category': 'int32'},
        chunksize=chunksize
    ):
        # Filter 1: Keep only interactions with valid items
        chunk = chunk[chunk['item_id'].isin(valid_items)]
        
        # Filter 2: Hash-based sampling
        # We keep user if user_id % 100 < (SAMPLE_RATE * 100)
        # This keeps exactly 10% of users, and it keeps ALL their records.
        chunk = chunk[chunk['user_id'] % 100 < (SAMPLE_RATE * 100)]
        
        chunks.append(chunk)
        print(f"Processed chunk... current accumulated rows: {sum(len(c) for c in chunks)}")

    # 3. Concatenate and Save
    full_sample = pd.concat(chunks, ignore_index=True)
    
    # Parse dates only at the end to save processing time
    full_sample['time'] = pd.to_datetime(full_sample['time'])
    full_sample['date'] = full_sample['time'].dt.date
    
    print(f"Final Sample Size: {len(full_sample)} rows")
    full_sample.to_parquet(OUTPUT_FILE, index=False)
    print(f"Saved to {OUTPUT_FILE}")

create_sample_dataset()

Reading raw data (Sample Rate: 10.0%)...
Item Subset Size: 6052459
Processed chunk... current accumulated rows: 5539
Processed chunk... current accumulated rows: 13399
Processed chunk... current accumulated rows: 20686
Processed chunk... current accumulated rows: 31035
Processed chunk... current accumulated rows: 38090
Processed chunk... current accumulated rows: 52333
Processed chunk... current accumulated rows: 61230
Processed chunk... current accumulated rows: 67666
Processed chunk... current accumulated rows: 77156
Processed chunk... current accumulated rows: 84391
Processed chunk... current accumulated rows: 90976
Processed chunk... current accumulated rows: 98515
Processed chunk... current accumulated rows: 105672
Processed chunk... current accumulated rows: 114319
Processed chunk... current accumulated rows: 122493
Processed chunk... current accumulated rows: 133179
Processed chunk... current accumulated rows: 142823
Processed chunk... current accumulated rows: 152929
Processed 

In [26]:
def ingest_data(RAW_ITEM_FILE):
    print("Ingesting raw text files...")
    
    df_item = pd.read_csv(
        RAW_ITEM_FILE, 
        header=None, 
        names=['item_id', 'item_geohash', 'item_category'],
        sep='\t',
        dtype={'item_id': 'int32', 'item_category': 'int32'}
    )

    # Save as Parquet (The "Silver" Layer)
    df_item.to_parquet(f"{DATA_PATH}/items.parquet", index=False)
    print(f"Data ingested and saved to Parquet. Items: {len(df_item)}")

if not os.path.exists(f"{DATA_PATH}/items.parquet"):
    ingest_data(RAW_ITEM_DATA_PATH)

def load_parquet_data():
    print("Loading from Parquet (Fast)...")
    df_item = pd.read_parquet(f"{DATA_PATH}/items.parquet")
    return df_item

df_item = load_parquet_data()

Ingesting raw text files...
Data ingested and saved to Parquet. Items: 6781009
Loading from Parquet (Fast)...


In [112]:
RAW_USERS = f"{DATA_PATH}/users.parquet"
RAW_ITEMS = f"{DATA_PATH}/items.parquet"

TRAIN_OUT = f"{DATA_PATH}/train_processed.parquet"
VAL_OUT = f"{DATA_PATH}/val_processed.parquet"
TEST_OUT = f"{DATA_PATH}/test_processed.parquet"

In [113]:
def extract_advanced_features(df, target_date_str, window_days=[1, 3]):

    target_ts = pd.Timestamp(target_date_str)
    target_date = target_ts.date()
    logger.info(f"\n--- Processing Target Date: {target_date} ---")
    
    # 1. Define History (Features) vs Target (Labels)
    history_df = df[df['date'] < target_date]
    target_df = df[df['date'] == target_date]
    
    # 2. Candidate Generation (The "Anchor")
    start_anchor = target_date - timedelta(days=1)
    candidates = history_df[history_df['date'] >= start_anchor][['user_id', 'item_id', 'item_category']].drop_duplicates()
    
    # 3. Helper: Calculate Statistics for a Window
    def get_window_stats(subset_df, prefix):
        # --- A. Item Features (Popularity) ---
        item_stats = subset_df.groupby(['item_id', 'behavior_type']).size().unstack(fill_value=0)
        # Ensure cols 1-4 exist
        for i in [1, 2, 3, 4]: 
            if i not in item_stats: item_stats[i] = 0
        item_stats.columns = [f'{prefix}_item_clk', f'{prefix}_item_fav', f'{prefix}_item_cart', f'{prefix}_item_buy']
        
        # Item Conversion Rate (Buy/Click)
        item_stats[f'{prefix}_item_cr'] = item_stats[f'{prefix}_item_buy'] / (item_stats[f'{prefix}_item_clk'] + 1)
        
        # Unique Users per Item
        item_uniq = subset_df.groupby('item_id')['user_id'].nunique().to_frame(f'{prefix}_item_uniq_users')
        item_stats = item_stats.join(item_uniq)

        # --- B. User Features (Activity) ---
        user_stats = subset_df.groupby(['user_id', 'behavior_type']).size().unstack(fill_value=0)
        for i in [1, 2, 3, 4]: 
            if i not in user_stats: user_stats[i] = 0
        user_stats.columns = [f'{prefix}_user_clk', f'{prefix}_user_fav', f'{prefix}_user_cart', f'{prefix}_user_buy']
        
        # User Conversion Rate
        user_stats[f'{prefix}_user_cr'] = user_stats[f'{prefix}_user_buy'] / (user_stats[f'{prefix}_user_clk'] + 1)

        # --- C. User-Item Interaction (Specific Interest) ---
        ui_stats = subset_df.groupby(['user_id', 'item_id', 'behavior_type']).size().unstack(fill_value=0)
        for i in [1, 2, 3, 4]: 
            if i not in ui_stats: ui_stats[i] = 0
        ui_stats.columns = [f'{prefix}_ui_clk', f'{prefix}_ui_fav', f'{prefix}_ui_cart', f'{prefix}_ui_buy']

        # --- D. User-Category Interaction (Broader Interest - Adapted from baseline) ---
        uc_stats = subset_df.groupby(['user_id', 'item_category', 'behavior_type']).size().unstack(fill_value=0)
        for i in [1, 2, 3, 4]: 
            if i not in uc_stats: uc_stats[i] = 0
        uc_stats.columns = [f'{prefix}_uc_clk', f'{prefix}_uc_fav', f'{prefix}_uc_cart', f'{prefix}_uc_buy']

        return item_stats, user_stats, ui_stats, uc_stats

    # 4. Sliding Window Loop (Last 1 day, Last 3 days)
    for days in window_days:
        start_date = target_date - timedelta(days=days)
        window_df = history_df[history_df['date'] >= start_date]
        
        prefix = f"last_{days}d"
        i_stats, u_stats, ui_stats, uc_stats = get_window_stats(window_df, prefix)
        
        # Merge sequentially
        # Note: We merge on different keys for different feature sets
        candidates = candidates.merge(i_stats, on='item_id', how='left').fillna(0)
        candidates = candidates.merge(u_stats, on='user_id', how='left').fillna(0)
        candidates = candidates.merge(ui_stats, on=['user_id', 'item_id'], how='left').fillna(0)
        candidates = candidates.merge(uc_stats, on=['user_id', 'item_category'], how='left').fillna(0)

    # 5. Time & Recency Features
    # Get the hour of the last interaction
    last_interaction = history_df.sort_values('time').groupby(['user_id', 'item_id']).tail(1)
    last_interaction['last_touch_hour'] = last_interaction['time'].dt.hour
    last_interaction['date'] = pd.to_datetime(last_interaction['date'])
    last_interaction['days_since'] = (target_ts - last_interaction['date']).dt.days
    
    candidates = candidates.merge(
        last_interaction[['user_id', 'item_id', 'last_touch_hour', 'days_since']], 
        on=['user_id', 'item_id'], 
        how='left'
    ).fillna(-1) 

    # 6. LABEL GENERATION (The Ground Truth)
    # We check if the user bought the item ON the target date
    buys_on_target = target_df[target_df['behavior_type'] == 4]
    
    # Create a set of (user, item) tuples that were bought
    positive_pairs = set(zip(buys_on_target['user_id'], buys_on_target['item_id']))
    
    # Apply Label: 1 if bought, 0 otherwise
    candidates['label'] = candidates.apply(
        lambda x: 1 if (x['user_id'], x['item_id']) in positive_pairs else 0, 
        axis=1
    )
    
    logger.info(f"Candidates: {len(candidates)} | Positives (Label=1): {candidates['label'].sum()}")
    return candidates

In [114]:
df_user = pd.read_parquet(RAW_USERS)
df_item = pd.read_parquet(RAW_ITEMS)

In [115]:
df_user.head()

,user_id,item_id,behavior_type,user_geohash,item_category,time,date
0,77413503,159477064,1,None,3846,2014-12-13 13:00:00,2014-12-13
1,77413503,392053776,1,None,3846,2014-12-13 20:00:00,2014-12-13
2,77413503,80129884,1,9rmq1tj,3846,2014-12-13 11:00:00,2014-12-13
3,77413503,360889319,1,9rmq6fb,3846,2014-12-13 10:00:00,2014-12-13
4,77413503,80129884,1,9rmq1tk,3846,2014-12-13 11:00:00,2014-12-13


In [117]:
def prepare_train_val_test_splits(RAW_USER_FILE, RAW_ITEM_FILE):

    logger.info("Loading data...")
    df_user = pd.read_parquet(RAW_USER_FILE)
    df_item = pd.read_parquet(RAW_ITEM_FILE)

    valid_items = set(df_item['item_id'])
    df_user = df_user[df_user['item_id'].isin(valid_items)]
    df_user['date'] = df_user['time'].dt.date

    logger.info("Generating Training set...")
    train_df = extract_advanced_features(df_user, "2014-12-08")
    train_df.to_parquet(TRAIN_OUT, index=False)

    logger.info("Generating Validation set...")
    val_df = extract_advanced_features(df_user, "2014-12-12")
    val_df.to_parquet(VAL_OUT, index=False)
    
    logger.info("Generating Test set...")
    test_df = extract_advanced_features(df_user, "2014-12-18")
    test_df.to_parquet(TEST_OUT, index=False)

    logger.info("All sets generated successfully!")
    return train_df, val_df, test_df

train_df, val_df, test_df = prepare_train_val_test_splits(RAW_USERS, RAW_ITEMS)


2025-11-27 10:17:23.399 | INFO     | __main__:prepare_train_val_test_splits:3 - Loading data...
2025-11-27 10:17:28.253 | INFO     | __main__:prepare_train_val_test_splits:11 - Generating Training set...
2025-11-27 10:17:28.258 | INFO     | __main__:extract_advanced_features:5 - 
--- Processing Target Date: 2014-12-08 ---
2025-11-27 10:17:30.831 | INFO     | __main__:extract_advanced_features:95 - Candidates: 81266 | Positives (Label=1): 245
2025-11-27 10:17:30.943 | INFO     | __main__:prepare_train_val_test_splits:15 - Generating Validation set...
2025-11-27 10:17:30.943 | INFO     | __main__:extract_advanced_features:5 - 
--- Processing Target Date: 2014-12-12 ---
2025-11-27 10:17:35.617 | INFO     | __main__:extract_advanced_features:95 - Candidates: 88911 | Positives (Label=1): 1756
2025-11-27 10:17:35.712 | INFO     | __main__:prepare_train_val_test_splits:19 - Generating Test set...
2025-11-27 10:17:35.713 | INFO     | __main__:extract_advanced_features:5 - 
--- Processing Targe

In [118]:
TRAIN_PATH = f"{DATA_PATH}/train_processed.parquet"
VAL_PATH = f"{DATA_PATH}/val_processed.parquet"
TEST_PATH = f"{DATA_PATH}/test_processed.parquet"

In [119]:
df_user.head()

,user_id,item_id,behavior_type,user_geohash,item_category,time,date
0,77413503,159477064,1,None,3846,2014-12-13 13:00:00,2014-12-13
1,77413503,392053776,1,None,3846,2014-12-13 20:00:00,2014-12-13
2,77413503,80129884,1,9rmq1tj,3846,2014-12-13 11:00:00,2014-12-13
3,77413503,360889319,1,9rmq6fb,3846,2014-12-13 10:00:00,2014-12-13
4,77413503,80129884,1,9rmq1tk,3846,2014-12-13 11:00:00,2014-12-13


In [120]:
def downsample(df, ratio=20):
    positives = df[df['label'] == 1]
    negatives = df[df['label'] == 0]
    
    if len(negatives) > len(positives) * ratio:
        negatives = negatives.sample(n=len(positives) * ratio, random_state=42)
    
    return pd.concat([positives, negatives]).sample(frac=1)

In [121]:

def evaluate_model(model, dataset_df, feature_cols, total_daily_buys=None, dataset_name="Validation"):

    preds = model.predict(dataset_df[feature_cols])

    pred_indices = (preds == 1)
    prediction_pairs = set(zip(dataset_df.loc[pred_indices, 'user_id'], dataset_df.loc[pred_indices, 'item_id']))

    # Actual Positives IN CANDIDATE SET: (User, Item) where label was 1
    actual_indices = (dataset_df['label'] == 1)
    reference_pairs = set(zip(dataset_df.loc[actual_indices, 'user_id'], dataset_df.loc[actual_indices, 'item_id']))

    # Intersection = True Positives found by the model
    intersection = prediction_pairs & reference_pairs
    tp_count = len(intersection)

    # Precision is always (TP / Predicted)
    precision = tp_count / len(prediction_pairs) if prediction_pairs else 0.0

    # Recall Logic
    if total_daily_buys:
        # GLOBAL RECALL: TP / All purchases in the world that day
        recall = tp_count / total_daily_buys
        metric_type = "Global"
    else:
        # LOCAL RECALL: TP / All purchases in our candidate set
        recall = tp_count / len(reference_pairs) if reference_pairs else 0.0
        metric_type = "Local"

    # F1 Calculation
    f1 = 0.0 if (precision + recall) == 0 else (2 * precision * recall) / (precision + recall)

    logger.info(f"[{dataset_name} - {metric_type}] Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    return precision, recall, f1

In [122]:
model_params = {
    "LogisticRegression": {
        "class_weight": 'balanced',
        "max_iter": 2000,
        "random_state": 42
    },
    "RandomForest": {
        "n_estimators": 50,
        "max_depth": 10,
        "random_state": 42
    },
    "XGBoost": {
        "n_estimators": 10,
        "max_depth": 3,
        "learning_rate": 0.1,
        "eval_metric": 'logloss',
        "random_state": 42
    }
}
def train_and_evaluate(model_params=model_params, mlflow_experiment=MLFLOW_EXPERIMENT_NAME):
    
    logger.info(f"Loading data from {DATA_PATH}...")
    train_df = pd.read_parquet(TRAIN_PATH)
    val_df = pd.read_parquet(VAL_PATH)
    test_df = pd.read_parquet(TEST_PATH)
    
    raw_df = pd.read_parquet(f"{DATA_PATH}/users.parquet")
    val_date = pd.Timestamp("2014-12-12").date()
    val_global_buys = len(raw_df[(raw_df['date'] == val_date) & (raw_df['behavior_type'] == 4)])

    test_date = pd.Timestamp("2014-12-18").date()
    test_global_buys = len(raw_df[(raw_df['date'] == test_date) & (raw_df['behavior_type'] == 4)])
    
    logger.info(f"Global Buys - Val: {val_global_buys}, Test: {test_global_buys}")
    
    # Separate Features and Target
    drop_cols = ['user_id', 'item_id', 'item_category', 'label']
    feature_cols = [c for c in train_df.columns if c not in drop_cols]
    
    # DOWNSAMPLING (Train set only)
    logger.info("Downsampling negative class in training set (1:20 ratio)...")
    train_balanced = downsample(train_df, ratio=20)
    X_train_bal = train_balanced[feature_cols]
    y_train_bal = train_balanced['label']

    signature = infer_signature(X_train_bal, y_train_bal)
    input_example = X_train_bal.head(1)

    mlflow.set_experiment(mlflow_experiment)
    best_model_info = {"f1": -1, "run_id": None, "description": None}
    
    def run_model(model_name, model_obj, params):
        nonlocal best_model_info
        with mlflow.start_run(run_name=model_name):
            logger.info(f"Training {model_name}...")
            model_obj.fit(X_train_bal, y_train_bal)
            
            # 1. Local Evaluation (Candidate Set)
            p_local, r_local, f1_local = evaluate_model(model_obj, val_df, feature_cols, total_daily_buys=None, dataset_name="Val_Local")
            
            # 2. Global Evaluation (Business Metric)
            p_global, r_global, f1_global = evaluate_model(model_obj, val_df, feature_cols, total_daily_buys=val_global_buys, dataset_name="Val_Global")
            
            mlflow.log_metrics({
                "val_f1_local": f1_local,
                "val_precision_local": p_local,
                "val_recall_local": r_local,
                "val_f1_global": f1_global, 
                "val_recall_global": r_global
            })
            mlflow.log_params({
                "model_type": model_name.split('_')[1], # e.g. "LogisticRegression"
                **params
            })
            mlflow.sklearn.log_model(model_obj, name="model", signature=signature, input_example=input_example)

            # Use Local F1 for model selection (since Global F1 is just a scaled version of Local F1 for the same candidate set)
            if f1_local > best_model_info["f1"]:
                active_run = mlflow.active_run()
                best_model_info = {
                    "f1": f1_local,
                    "run_id": active_run.info.run_id if active_run else None,
                    "description": model_name
                }

    # Run Baselines
    run_model("baseline_logistic_regression", LogisticRegression(**model_params["LogisticRegression"]), model_params["LogisticRegression"])
    run_model("baseline_random_forest", RandomForestClassifier(**model_params["RandomForest"]), model_params["RandomForest"])
    run_model("baseline_xgboost", xgb.XGBClassifier(**model_params["XGBoost"]), model_params["XGBoost"])

    def optuna_objective(trial):
        tuned_params = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 500),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "subsample": trial.suggest_float("subsample", 0.4, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
            "gamma": trial.suggest_float("gamma", 0.0, 5.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1e-1, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
            "eval_metric": "logloss",
            "random_state": 42
        }
        model = xgb.XGBClassifier(**tuned_params)
        model.fit(X_train_bal, y_train_bal)
        preds = model.predict(val_df[feature_cols])
        return f1_score(val_df['label'], preds)

    logger.info("Running Optuna tuning for XGBoost...")
    study = optuna.create_study(direction="maximize")
    study.optimize(optuna_objective, n_trials=30, show_progress_bar=False)

    best_params = {
        **study.best_params,
        "eval_metric": "logloss",
        "random_state": 42
    }

    with mlflow.start_run(run_name="xgboost_optuna_tuned"):
        tuned_model = xgb.XGBClassifier(**best_params)
        tuned_model.fit(X_train_bal, y_train_bal)

        # Validation Metrics (Local & Global)
        p_val_local, r_val_local, f1_val_local = evaluate_model(tuned_model, val_df, feature_cols, total_daily_buys=None, dataset_name="Val_Local")
        p_val_global, r_val_global, f1_val_global = evaluate_model(tuned_model, val_df, feature_cols, total_daily_buys=val_global_buys, dataset_name="Val_Global")
        
        # Test Metrics (Local & Global) - This is your FINAL reported score
        p_test_local, r_test_local, f1_test_local = evaluate_model(tuned_model, test_df, feature_cols, total_daily_buys=None, dataset_name="Test_Local")
        p_test_global, r_test_global, f1_test_global = evaluate_model(tuned_model, test_df, feature_cols, total_daily_buys=test_global_buys, dataset_name="Test_Global")

        mlflow.log_params({
            "model_type": "XGBoost_Optuna",
            **best_params
        })
        mlflow.log_metrics({
            "val_f1_local": f1_val_local,
            "val_f1_global": f1_val_global,
            "test_f1_local": f1_test_local,
            "test_f1_global": f1_test_global,
            "best_trial": study.best_trial.number
        })
        mlflow.sklearn.log_model(tuned_model, name="model", signature=signature, input_example=input_example)

        if f1_val_local > best_model_info["f1"]:
            active_run = mlflow.active_run()
            best_model_info = {
                "f1": f1_val_local,
                "run_id": active_run.info.run_id if active_run else None,
                "description": "xgboost_optuna_tuned"
            }

    if best_model_info["run_id"]:
        logger.info(f"Registering best model ({best_model_info['description']}) with val F1={best_model_info['f1']:.4f}")
        mlflow.register_model(f"runs:/{best_model_info['run_id']}/model", "mobile_item_rec_model")
    else:
        logger.warning("No model run recorded for registration.")

In [131]:
train_and_evaluate(model_params=model_params, mlflow_experiment=MLFLOW_EXPERIMENT_NAME)

2025-11-27 10:45:44.023 | INFO     | __main__:train_and_evaluate:22 - Loading data from /Users/duc.tran/mobile-item-recommendation/data/raw...
2025-11-27 10:45:44.995 | INFO     | __main__:train_and_evaluate:34 - Global Buys - Val: 7415, Test: 1952
2025-11-27 10:45:44.996 | INFO     | __main__:train_and_evaluate:41 - Downsampling negative class in training set (1:20 ratio)...
/Users/duc.tran/mobile-item-recommendation/venv/lib/python3.13/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values.

🏃 View run baseline_logistic_regression at: http://localhost:5000/#/experiments/1/runs/2fa9a13bdfa44e6a98bf4c32aeb90ed4
🧪 View experiment at: http://localhost:5000/#/experiments/1


OSError: [Errno 30] Read-only file system: '/app'

In [133]:
# Print predictions on test_df
logger.info("=" * 80)
logger.info("PREDICTIONS ON TEST SET")
logger.info("=" * 80)

# Load the best model from MLflow
from mlflow.tracking import MlflowClient

client = MlflowClient()
model_name = "mobile_item_rec_model"

loaded_model = mlflow.sklearn.load_model("/Users/duc.tran/mobile-item-recommendation/notebooks/mlruns/916499331672477983/models/m-92fb2d2b3d1f49ed8f728a6635120e1e/artifacts/")

# Load test data
test_df = pd.read_parquet(TEST_PATH)
drop_cols = ['user_id', 'item_id', 'item_category', 'label']
feature_cols = [c for c in test_df.columns if c not in drop_cols]

# Make predictions
logger.info(f"Making predictions on {len(test_df)} test samples...")
test_predictions = loaded_model.predict(test_df[feature_cols])
test_probabilities = loaded_model.predict_proba(test_df[feature_cols])[:, 1]

# Add predictions to test_df
test_df['prediction'] = test_predictions
test_df['buy_probability'] = test_probabilities

# Print summary statistics
logger.info("\n" + "=" * 80)
logger.info("PREDICTION SUMMARY")
logger.info("=" * 80)
logger.info(f"Total test samples: {len(test_df)}")
logger.info(f"Actual positives (label=1): {test_df['label'].sum()}")
logger.info(f"Predicted positives (prediction=1): {test_predictions.sum()}")
logger.info(f"True Positives: {((test_df['label'] == 1) & (test_predictions == 1)).sum()}")
logger.info(f"False Positives: {((test_df['label'] == 0) & (test_predictions == 1)).sum()}")
logger.info(f"False Negatives: {((test_df['label'] == 1) & (test_predictions == 0)).sum()}")
logger.info(f"True Negatives: {((test_df['label'] == 0) & (test_predictions == 0)).sum()}")

logger.info(f"\nProbability Statistics:")
logger.info(f"  Min probability: {test_probabilities.min():.4f}")
logger.info(f"  Max probability: {test_probabilities.max():.4f}")
logger.info(f"  Mean probability: {test_probabilities.mean():.4f}")
logger.info(f"  Median probability: {np.median(test_probabilities):.4f}")

# Show samples of predictions
logger.info("\n" + "=" * 80)
logger.info("SAMPLE PREDICTIONS - TRUE POSITIVES (Correctly Predicted Buys)")
logger.info("=" * 80)
true_positives = test_df[(test_df['label'] == 1) & (test_predictions == 1)]
if len(true_positives) > 0:
    print(true_positives[['user_id', 'item_id', 'item_category', 'label', 'prediction', 'buy_probability']].head(20))
    logger.info(f"\nShowing {min(20, len(true_positives))} of {len(true_positives)} true positives")
else:
    logger.info("No true positives found")

logger.info("\n" + "=" * 80)
logger.info("SAMPLE PREDICTIONS - FALSE NEGATIVES (Missed Buys)")
logger.info("=" * 80)
false_negatives = test_df[(test_df['label'] == 1) & (test_predictions == 0)]
if len(false_negatives) > 0:
    # Sort by probability to see which ones were close
    false_negatives_sorted = false_negatives.sort_values('buy_probability', ascending=False)
    print(false_negatives_sorted[['user_id', 'item_id', 'item_category', 'label', 'prediction', 'buy_probability']].head(20))
    logger.info(f"\nShowing {min(20, len(false_negatives))} of {len(false_negatives)} false negatives (sorted by probability)")
else:
    logger.info("No false negatives found")

logger.info("\n" + "=" * 80)
logger.info("SAMPLE PREDICTIONS - FALSE POSITIVES (Incorrectly Predicted Buys)")
logger.info("=" * 80)
false_positives = test_df[(test_df['label'] == 0) & (test_predictions == 1)]
if len(false_positives) > 0:
    # Sort by probability to see which ones had high confidence
    false_positives_sorted = false_positives.sort_values('buy_probability', ascending=False)
    print(false_positives_sorted[['user_id', 'item_id', 'item_category', 'label', 'prediction', 'buy_probability']].head(20))
    logger.info(f"\nShowing {min(20, len(false_positives))} of {len(false_positives)} false positives (sorted by probability)")
else:
    logger.info("No false positives found")

logger.info("\n" + "=" * 80)
logger.info("TOP PREDICTIONS BY PROBABILITY (All predictions sorted by buy_probability)")
logger.info("=" * 80)
top_predictions = test_df.nlargest(30, 'buy_probability')
print(top_predictions[['user_id', 'item_id', 'item_category', 'label', 'prediction', 'buy_probability']])

logger.info("\n" + "=" * 80)
logger.info("PREDICTIONS FOR ACTUAL BUYERS (label=1) - Sorted by Probability")
logger.info("=" * 80)
actual_buyers = test_df[test_df['label'] == 1].sort_values('buy_probability', ascending=False)
print(actual_buyers[['user_id', 'item_id', 'item_category', 'label', 'prediction', 'buy_probability']].head(30))

logger.info("\n" + "=" * 80)
logger.info("PREDICTIONS COMPLETE")
logger.info("=" * 80)


2025-11-27 10:47:51.032 | INFO     | __main__:<module>:2 - ================================================================================
2025-11-27 10:47:51.045 | INFO     | __main__:<module>:3 - PREDICTIONS ON TEST SET
2025-11-27 10:47:51.057 | INFO     | __main__:<module>:4 - ================================================================================
2025-11-27 10:47:51.559 | INFO     | __main__:<module>:20 - Making predictions on 75927 test samples...
2025-11-27 10:47:52.061 | INFO     | __main__:<module>:29 - 
2025-11-27 10:47:52.074 | INFO     | __main__:<module>:30 - PREDICTION SUMMARY
2025-11-27 10:47:52.077 | INFO     | __main__:<module>:31 - ================================================================================
2025-11-27 10:47:52.080 | INFO     | __main__:<module>:32 - Total test samples: 75927
2025-11-27 10:47:52.105 | INFO     | __main__:<module>:33 - Actual positives (label=1): 261
2025-11-27 10:47:52.131 | INFO     | __main__:<module>:34 - Predicted posi

         user_id    item_id  item_category  label  prediction  buy_probability
2433    59409308  236514690          11991      1           1         0.552270
3979    78679808  108523444           6648      1           1         0.545513
6514    48643708  396027419          12553      1           1         0.656751
6527    48643708  331484286           3690      1           1         0.519691
10352  129163002  165101425           5993      1           1         0.569846
10355  129163002  366320763           1686      1           1         0.564289
15550   16296509  239395596          11991      1           1         0.508897
20393   31356607  354525367           3942      1           1         0.569682
24038   94518207  287824233          12326      1           1         0.689332
25928  113587402  311685580           3942      1           1         0.653627
28587   96141405  328312925           7572      1           1         0.523936
29621   20229000  270664611           5480      1   

2025-11-27 10:47:52.618 | INFO     | __main__:<module>:83 - ================================================================================
2025-11-27 10:47:53.680 | INFO     | __main__:<module>:87 - 
2025-11-27 10:47:53.761 | INFO     | __main__:<module>:88 - PREDICTIONS FOR ACTUAL BUYERS (label=1) - Sorted by Probability
2025-11-27 10:47:53.778 | INFO     | __main__:<module>:89 - ================================================================================


         user_id    item_id  item_category  label  prediction  buy_probability
23332  140196509  315119130          12096      0           1         0.897175
9275   102961104   79329525          10895      0           1         0.896901
17736   63014409    7621463           9307      0           1         0.891091
20586   65862109   93657183           3942      0           1         0.889717
47256    1659408  151705313          12158      0           1         0.888716
65462  113844402  401890785           9614      0           1         0.880069
9281   102961104  114450090          14047      0           1         0.869505
69096     427500  113202187           9334      0           1         0.869382
17759   63014409   89506164           9232      0           1         0.869221
64558   55225001  368321478          10431      0           1         0.859529
58208    6223506  318556386          11991      0           1         0.852175
17747   63014409   25810762          10412      0   

2025-11-27 10:47:54.127 | INFO     | __main__:<module>:93 - 
2025-11-27 10:47:54.211 | INFO     | __main__:<module>:94 - PREDICTIONS COMPLETE


         user_id    item_id  item_category  label  prediction  buy_probability
47896    9439407  138880554           9205      1           1         0.840472
36170   57892506  294167986           8792      1           1         0.839614
36491   42385801  291385196           1963      1           1         0.823334
58032   88574400  276458769           1032      1           1         0.796867
46828    8884009  355024510           3368      1           1         0.785616
43274   29745002  146714345           3925      1           1         0.773062
54266   29093004  388669975           2029      1           1         0.771537
36694    8646903  305613858           3660      1           1         0.763107
30364   81799304  240505417           6417      1           1         0.694920
24038   94518207  287824233          12326      1           1         0.689332
31771   29010009   14136232          10662      1           1         0.659429
6514    48643708  396027419          12553      1   

2025-11-27 10:47:54.417 | INFO     | __main__:<module>:95 - ================================================================================
